In [98]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
input_dir = 'input'
print(os.listdir(input_dir))

['sample_submission.csv', 'train_identity.csv', 'train_transaction.csv']


In [99]:
from sklearn import preprocessing
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import roc_auc_score,precision_recall_curve,roc_curve
import matplotlib.pyplot as plt
import imblearn #for SMOTE
from sklearn.decomposition import PCA, KernelPCA

In [100]:
%%time
# From kernel https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
# WARNING! THIS CAN DAMAGE THE DATA 
def reduce_mem_usage2(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
#         else:
#             df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

Wall time: 0 ns


In [101]:
%%time
def train_xgb(trainx,trainy):
    clf = xgb.XGBClassifier(n_estimators=500,
                        n_jobs=-1,
                        max_depth=9,
                        learning_rate=0.05,
                        subsample=0.9,
                        colsample_bytree=0.9,
                        missing=-999,
                        #tree_method='gpu_hist'
                        )

    clf.fit(trainx,trainy)
    return clf
from sklearn.linear_model import LogisticRegression

def train_lbg(trainx,trainy):
    params={'learning_rate': 0.01,
            'objective': 'binary',
            'metric': 'auc',
            'num_threads': -1,
            'num_leaves': 256,
            'verbose': 1,
            'random_state': 42,
            'bagging_fraction': 1,
            'feature_fraction': 0.85
           }
    clf = lgb.LGBMClassifier(**params, n_estimators=3000)
    clf.fit(trainx,trainy)
    return clf

def train_logistic(trainx,trainy):
    lr=LogisticRegression(max_iter=10000)
    lr.fit(X_train,y_train)
    return lr

Wall time: 0 ns


In [102]:
#Plotting
from sklearn.metrics import auc
def plot_pre_curve(y_test,probs):
    precision, recall, thresholds = precision_recall_curve(y_test, probs)
#     fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
    auc_pr = auc(recall,precision)
    plt.plot([0, 1], [0.5, 0.5], linestyle='--')
    # plot the precision-recall curve for the model
    plt.plot(recall, precision, marker='.')
    plt.title("precision recall curve "+str(auc_pr))
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # show the plot
    plt.show()
    
def plot_roc(y_test,probs):
    fpr, tpr, thresholds = roc_curve(y_test, probs)
    # plot no skill
    plt.plot([0, 1], [0, 1], linestyle='--')
    # plot the roc curve for the model
    plt.plot(fpr, tpr, marker='.')
    plt.title("ROC curve "+str(roc_auc_score(y_test,probs)))
    plt.xlabel('false positive rate')
    plt.ylabel('true positive rate')
    # show the plot
    plt.show()

In [103]:
def predict_plot(model):
    print(y_test.shape,X_test.shape)
    predictions =model.predict_proba(X_test)[:,1]
    print(predictions)
    plot_pre_curve(y_test,predictions)
    plot_roc(y_test,predictions)


In [104]:
%%time
train_transaction = pd.read_csv(input_dir+'/train_transaction.csv', index_col='TransactionID',nrows=160000)
# test_transaction = pd.read_csv(input_dir+'/test_transaction.csv', index_col='TransactionID')

train_identity = pd.read_csv(input_dir+'/train_identity.csv', index_col='TransactionID')
# test_identity = pd.read_csv(input_dir+'/test_identity.csv', index_col='TransactionID')

sample_submission = pd.read_csv(input_dir+'/sample_submission.csv', index_col='TransactionID')

Wall time: 5.65 s


In [105]:
%%time
from sklearn.model_selection import train_test_split

train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
# test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)
train = train.fillna(-999)
# m_features = 
train = reduce_mem_usage2(train)

print(train.shape)
# print(test.shape)



# Drop target, fill in NaNs
# X_train = train.drop('isFraud', axis=1)
# X_test = test.copy()


Memory usage of dataframe is 534.79 MB
Memory usage after optimization is: 174.22 MB
Decreased by 67.4%
(160000, 433)
Wall time: 37.9 s


In [106]:
# Label Encoding
for f in X_train.columns:
    if X_train[f].dtype=='object' or X_test[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f].values) + list(X_test[f].values))
        X_train[f] = lbl.transform(list(X_train[f].values))
        X_test[f] = lbl.transform(list(X_test[f].values))   

In [107]:
import datetime

START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
train['Date'] = train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
train['_weekday'] = train['Date'].dt.dayofweek
train['_hour'] = train['Date'].dt.hour
train['_day'] = train['Date'].dt.day

train['_weekday'] = train['_weekday'].astype(str)
train['_hour'] = train['_hour'].astype(str)
train['_weekday__hour'] = train['_weekday'] + train['_hour']

cnt_day = train['_day'].value_counts()
cnt_day = cnt_day / cnt_day.mean()
train['_count_rate'] = train['_day'].map(cnt_day.to_dict())

train.drop(['TransactionDT','Date','_day'], axis=1, inplace=True)

In [108]:
def pca_vcols(all_data):
    vcols = [f'V{i}' for i in range(1,340)]
    sc = preprocessing.MinMaxScaler()

    pca = PCA(n_components=2) #0.99
    vcol_pca = pca.fit_transform(sc.fit_transform(all_data[vcols].fillna(-1)))

    all_data['_vcol_pca0'] = vcol_pca[:,0]
    all_data['_vcol_pca1'] = vcol_pca[:,1]
    all_data['_vcol_nulls'] = all_data[vcols].isnull().sum(axis=1)

    all_data.drop(vcols, axis=1, inplace=True)
pca_vcols(train)

In [109]:
def extract_amount_features(all_data):
    import re
    #strong id features
    all_data['_P_emaildomain__addr1'] = all_data['P_emaildomain'].astype(str) + '__' + all_data['addr1'].astype(str)
    all_data['_card1__card2'] = all_data['card1'].astype(str) + '__' + all_data['card2'].astype(str)
    all_data['_card1__addr1'] = all_data['card1'].astype(str) + '__' + all_data['addr1'].astype(str)
    all_data['_card2__addr1'] = all_data['card2'].astype(str) + '__' + all_data['addr1'].astype(str)
    all_data['_card12__addr1'] = all_data['_card1__card2'] + '__' + all_data['addr1'].astype(str)
    all_data['_card_all__addr1'] = all_data['_card1__card2'] + '__' + all_data['addr1'].astype(str)
    
    #amount features
    all_data['_amount_decimal'] = ((all_data['TransactionAmt'] - all_data['TransactionAmt'].astype(int)) * 1000).astype(int)
    all_data['_amount_decimal_len'] = all_data['TransactionAmt'].apply(lambda x: len(re.sub('0+$', '', str(x)).split('.')[1]))
    all_data['_amount_fraction'] = all_data['TransactionAmt'].apply(lambda x: float('0.'+re.sub('^[0-9]|\.|0+$', '', str(x))))
    all_data[['TransactionAmt','_amount_decimal','_amount_decimal_len','_amount_fraction']].head(10)
    
    
    # statistics based on several features (how thery were chosen?? TODO)
    cols = ['ProductCD','card1','card2','card5','card6','P_emaildomain','_card_all__addr1']
    #,'card3','card4','addr1','dist2','R_emaildomain'

    # amount mean&std
    for f in cols:
        all_data[f'_amount_mean_{f}'] = all_data['TransactionAmt'] / all_data.groupby([f])['TransactionAmt'].transform('mean')
        all_data[f'_amount_std_{f}'] = all_data['TransactionAmt'] / all_data.groupby([f])['TransactionAmt'].transform('std')
        all_data[f'_amount_pct_{f}'] = (all_data['TransactionAmt'] - all_data[f'_amount_mean_{f}']) / all_data[f'_amount_std_{f}']

    # freq encoding
    for f in cols:
        vc = all_data[f].value_counts(dropna=False)
        all_data[f'_count_{f}'] = all_data[f].map(vc)
    
    # handle object columns    
    cat_cols = [f'id_{i}' for i in range(12,39)]
    for i in cat_cols:
        if i in all_data.columns:
            all_data[i] = all_data[i].astype(str)
            all_data[i].fillna('unknown', inplace=True)

    enc_cols = []
    for i, t in all_data.loc[:, all_data.columns != 'isFraud'].dtypes.iteritems():
        if t == object:
            enc_cols.append(i)
            #df = pd.concat([df, pd.get_dummies(df[i].astype(str), prefix=i)], axis=1)
            #df.drop(i, axis=1, inplace=True)
            all_data[i] = pd.factorize(all_data[i])[0]
            #all_data[i] = all_data[i].astype('category')
    
extract_amount_features(train)

In [110]:
# def deleteStartingWith(df,prefix):
#     cols= df.loc[:, df.columns.str.startswith(prefix)]
#     del cols
#     del df[[col for col in df if col.startswith(prefix)]]
# deleteStartingWith(X_train,'M')
# deleteStartingWith(X_test,'M')
# deleteStartingWith(X_train,'D')
# deleteStartingWith(X_test,'D')

# del X_test[[col for col in X_train if col.startswith('M')]]

In [111]:
y_train = train['isFraud'].copy()

X_train,X_test,y_train,y_test=train_test_split(train.drop('isFraud',axis=1),y_train,test_size=.2,random_state=1)

from imblearn.over_sampling import SMOTE
#ratio='minority'
smote = SMOTE(n_jobs=8,sampling_strategy='minority')
X_sm, y_sm = smote.fit_sample(X_train, y_train)
X_sm.shape,y_sm.shape,X_train.shape

In [112]:
X_train.columns

Index(['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card4',
       'card5', 'card6', 'addr1', 'addr2',
       ...
       '_amount_mean__card_all__addr1', '_amount_std__card_all__addr1',
       '_amount_pct__card_all__addr1', '_count_ProductCD', '_count_card1',
       '_count_card2', '_count_card5', '_count_card6', '_count_P_emaildomain',
       '_count__card_all__addr1'],
      dtype='object', length=136)

In [ ]:
%%time
clf_lgb = train_lbg(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.85, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.85
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Warning] feature_fraction is set=0.85, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.85
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] num_threads is set=-1, n_jobs=-1 will be ignored. Current value: num_threads=-1
[LightGBM] [Info] Number of positive: 3512, number of negative: 124488
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.117286 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18947
[LightGBM] [Info] Number of data points in the tr

In [ ]:
# clf.predict_proba(X_test)
predict_plot(clf_lgb)
# predict_plot(clf)

In [ ]:
%%time
clf = train_xgb(X_train, y_train)
# clf = train_logistic(X_train,y_train)

In [ ]:
predict_plot(clf)

%%time
clf_smote = train_xgb(X_sm,y_sm)

predict_plot(clf_smote)

In [ ]:
# sample_submission['isFraud'] = clf.predict_proba(X_test)[:,1]
# sample_submission.to_csv('simple_xgboost.csv')

In [ ]:
# def train_and_evaluate(X_train,y_train,X_test,y_test):
#     clf = xgb.XGBClassifier(n_estimators=500,
#                         n_jobs=4,
#                         max_depth=9,
#                         learning_rate=0.05,
#                         subsample=0.9,
#                         colsample_bytree=0.9,
#                         missing=-999)

#     clf.fit(X_train, y_train)
#     print(y_test.shape,X_test.shape)
#     predictions =clf.predict(X_test)
#     plot_pre_curve(y_test,predictions)
#     plot_roc(y_test,predictions)